In [1]:
import os
import sys
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

PROJECT_ROOT = r"D:\demand_forecasting_system"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from tasks.extract_mssql import fetch_table_data

In [2]:
df_order_items=fetch_table_data('blinkit_order_items')


D:\demand_forecasting_system\src\tasks\extract_mssql.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-10-21 21:08:11,143 | INFO | data_pipeline | Fetched 5000 rows from table 'blinkit_order_items' (DB)


Keep only essential columns

In [4]:
usable_cols = ['order_id', 'product_id', 'quantity', 'unit_price']
df_order_items = df_order_items[usable_cols].copy()


Remove duplicates

In [5]:
df_order_items.drop_duplicates(subset=['order_id', 'product_id'], inplace=True)


Handle missing values

In [6]:
df_order_items['quantity'] = df_order_items['quantity'].fillna(0)
df_order_items['unit_price'] = df_order_items['unit_price'].fillna(0)
df_order_items.dropna(subset=['order_id', 'product_id'], inplace=True)


Convert datatypes

In [7]:
df_order_items['order_id'] = df_order_items['order_id'].astype(int)
df_order_items['product_id'] = df_order_items['product_id'].astype(int)
df_order_items['quantity'] = df_order_items['quantity'].astype(int)
df_order_items['unit_price'] = df_order_items['unit_price'].astype(float)


Derived columns

In [8]:
df_order_items['total_price'] = df_order_items['quantity'] * df_order_items['unit_price']


In [9]:
print(df_order_items.isnull().sum())
print(df_order_items.dtypes)
print(df_order_items.shape)


order_id       0
product_id     0
quantity       0
unit_price     0
total_price    0
dtype: int64
order_id         int64
product_id       int64
quantity         int64
unit_price     float64
total_price    float64
dtype: object
(5000, 5)


In [10]:
save_path = r"D:\demand_forecasting_system\data\processed\blinkit_order_items_clean.csv"
import os
os.makedirs(os.path.dirname(save_path), exist_ok=True)
df_order_items.to_csv(save_path, index=False)
print(f"✅ Order items data cleaned and saved to: {save_path}")


✅ Order items data cleaned and saved to: D:\demand_forecasting_system\data\processed\blinkit_order_items_clean.csv
